# Example state transfer

**Objective:** Create a model of `A(➊➋➌) <-> B(➊➋) + C(➌) <-> A(➌➊➋)` demonstrating state transfer and network expansion.

In the following, we will create a model of stable isotope labeling in a simple reaction network, demonstrate how to encode atom mappings, and how to create observables.


## Model definition

In [ ]:
from isrene import ureg, Q_
from isrene.model_definition import *
from isrene.model_definition.model_export_sbml import ModelExporterSbmlMulti
from isrene.sbml_multi.sbml_multi_expand import NetworkGenerator

import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np

In [ ]:
model = Model("example_state_transfer")

compartment = Compartment("c", ureg.Quantity(1, ureg.mL), model=model)

# species types
# Metabolites that can be labeled (second argument is the number of labelable sites)
A = Metabolite("A", 3, model=model)
B = Metabolite("B", 2, model=model)
C = Metabolite("C", 1, model=model)
# Enzyme (not labeled)
E = Enzyme("E", name="enzyme", model=model)

# Reactions
# modular rate law version
mrl_version = "hal"
# A(➊➋➌) <-> B(➊➋) + C(➌)
AtomMappingReaction(
    "R1",
    [(A(compartment=compartment), "abc")],
    [(B(compartment=compartment), "ab"), (C(compartment=compartment), "c")],
    reversible=True,
    enzyme=E(compartment=compartment),
    rate_law_generator=ModularRateLawGenerator(version=mrl_version),
)
# B(➊➋) + C(➌) <-> A(➌➊➋)
AtomMappingReaction(
    "R2",
    [(B(compartment=compartment), "ab"), (C(compartment=compartment), "c")],
    [(A(compartment=compartment), "cab")],
    reversible=True,
    enzyme=E(compartment=compartment),
    rate_law_generator=ModularRateLawGenerator(version=mrl_version),
)

# Initial concentrations
# A(⚫⚫⚫)
InitialConcentration(
    A.labeled(compartment=compartment), ureg.Quantity(1.0, ureg.mM)
)
# B(⚪⚪)
InitialConcentration(
    B.unlabeled(compartment=compartment), ureg.Quantity(1.0, ureg.mM)
)
# C(⚪)
InitialConcentration(
    C.unlabeled(compartment=compartment), ureg.Quantity(0.0, ureg.mM)
)
# E()
InitialConcentration(
    E(compartment=compartment), Q_(1, model.concentration_unit)
)

# Assume we can observe labeling in `C`
# .. and have raw mass spec intensities
RawMdvObservable(
    A, compartments=[compartment], sum_formula="C6H12O3NSi", model=model
)
# .. or our measurements are already corrected for natural isotope abundance
MdvObservable(A, compartments=[compartment], model=model);

In [ ]:
# show what's inside the model
model.print()

## Write model to SBML multi and expand to SBML core

In [ ]:
# write SBML multi
multi_file = f"tmp/{model.id}_multi.xml"
exporter = ModelExporterSbmlMulti(model)
exporter.export(multi_file)

# expand network and save SBML core model
core_model_name = f"tmp/{model.id}_core.xml"
nwg = NetworkGenerator(model=multi_file)
nwg.create_sbml_core_model(core_model_name)

Let's look at the generated model. All relevant isotoplogue species and reactions have been generated:

In [ ]:
import libsbml
import re

# load generated SBML-core model
sbml_reader = libsbml.SBMLReader()
sbml_doc = sbml_reader.readSBML(core_model_name)
sbml_model = sbml_doc.getModel()


def pretty_species(species_id) -> str:
    """Format species names nicely"""
    res = re.sub(r"_C\d_l", "⚫", re.sub(r"_C\d_u", "⚪", species_id))
    if res.endswith("_X__"):
        return res.replace("_X__", ")").replace("_c", "(")
    return res


print("⚪ unlabeled (12C)")
print("⚫ labeled   (13C)")
print()

print("Species:")
for species in sorted(s.getId() for s in sbml_model.getListOfSpecies()):
    print(f"{pretty_species(species).ljust(10)} ({species})")


print("\nReactions:")
for reaction in sbml_model.getListOfReactions():
    reactants = " + ".join(
        [
            "%s %s"
            % (
                int(r.getStoichiometry()) if r.getStoichiometry() > 1 else "",
                pretty_species(r.getSpecies()),
            )
            for r in reaction.getListOfReactants()
        ]
    )
    products = " + ".join(
        [
            "%s %s"
            % (
                int(r.getStoichiometry()) if r.getStoichiometry() > 1 else "",
                pretty_species(r.getSpecies()),
            )
            for r in reaction.getListOfProducts()
        ]
    )
    reversible = "<" if reaction.getReversible() else ""
    """
    print(
        '%3s: %10s %1s->%10s\t\t[%s]' % (
            reaction.getId(),
            reactants,
            reversible,
            products,
            libsbml.formulaToL3String(reaction.getKineticLaw().getMath())
        )
    )
    """
    print(
        "  %3s: %10s %1s->%10s"
        % (
            reaction.getId(),
            reactants,
            reversible,
            products,
        )
    )

Also the rate expression have been adapted to the individual isotopologue reactions:

In [ ]:
import sympy as sp

sp.init_printing()
from sbmlmath import sbml_math_to_sympy

print("\nRate expressions:")
for reaction in sbml_model.getListOfReactions():
    print(reaction.getId())
    # sp.pprint(sbml_math_to_sympy(reaction.getKineticLaw().getMath()))
    print("   ", libsbml.formulaToL3String(reaction.getKineticLaw().getMath()))
    print()

## Import to AMICI

The SBML core model can be import by AMICI:

In [ ]:
from amici.sbml_import import SbmlImporter, assignmentRules2observables

amici_model_name = model.id
amici_model_dir = f"tmp/{amici_model_name}"

sbml_importer = SbmlImporter(core_model_name)
observables = assignmentRules2observables(
    sbml_importer.sbml,
    lambda parameter: parameter.getId().startswith("observable_"),
)
# strip 'observable_' prefix
observables = {
    k[len("observable_") :]: v
    for k, v in observables.items()
    if k.startswith("observable_")
}

sbml_importer.sbml2amici(
    model_name=amici_model_name,
    output_dir=amici_model_dir,
    verbose=False,
    observables=observables,
)

In [ ]:
pprint(observables)

## Simulate the model

In [ ]:
import amici
from pprint import pprint

model_module = amici.import_model_module(
    amici_model_name, module_path=amici_model_dir
)
amici_model = model_module.getModel()

print("Parameters:")
pprint(amici_model.getParameterIds())
print("States:")
pprint(amici_model.getStateIds())
print("Obsevables:")
pprint(amici_model.getObservableIds())

In [ ]:
# Set model parameters
amici_model.setParametersByIdRegex(".*", 1.0)
amici_model.setParameterById("k_eq_R1", 0.0001)
amici_model.setParameterById("k_eq_R2", 1)
amici_model.setParameterById("k_r_v_R1", 0.01)
amici_model.setParameterById("k_r_v_R2", 0.001)

amici_model.setTimepoints(list(range(1500)))
solver = amici_model.getSolver()
rdata = amici.runAmiciSimulation(amici_model, solver)

# Visualize isotopologue trajectories for each labeled metabolite
from amici.plotting import plot_state_trajectories, plot_observable_trajectories

for metabolite in "ABC":
    state_indices = [
        i
        for i, name in enumerate(amici_model.getStateNames())
        if name.startswith(f"{metabolite} in ")
    ]
    plot_state_trajectories(
        rdata, model=amici_model, state_indices=state_indices
    )
    plt.title(f"State trajectories - {metabolite}")

In [ ]:
# Visualize observable MDVs, corrected for natural isotope abundance and uncorrected
plot_observable_trajectories(
    rdata,
    model=amici_model,
    observable_indices=[
        i
        for i, n in enumerate(amici_model.getObservableIds())
        if n.startswith("mdv_A_")
    ],
)
plt.ylabel("Relative abundance")
plt.title("Corrected MDV");

In [ ]:
plot_observable_trajectories(
    rdata,
    model=amici_model,
    observable_indices=[
        i
        for i, n in enumerate(amici_model.getObservableIds())
        if n.startswith("mdv_raw_A")
    ],
)
plt.ylabel("Intensity")
plt.title("Raw MDV");

In [ ]:
# Verify that corrected and raw observables match
mid_corrected = np.vstack(
    [
        rdata.by_id(f"mdv_A_M{i}")
        for i in range(model["observable_mdv_A"].max_labels + 1)
    ]
)
mid_raw = np.vstack(
    [
        rdata.by_id(f"mdv_raw_A_174_M{i}")
        for i in range(model["observable_mdv_raw_A"].num_measurements)
    ]
)
corr = model["observable_mdv_raw_A"].get_correction_matrix()
assert np.all(
    np.abs(
        np.dot(corr, mid_corrected[:, -1])
        - mid_raw[:, -1] / np.sum(mid_raw[:, -1])
        < 1e-2
    )
)

In [ ]:
# Check that corrected observables match expectations
m0 = rdata.by_id("A_c_C1_u_C2_u_C3_u")[-1]
m1 = (
    rdata.by_id("A_c_C1_u_C2_u_C3_l")[-1]
    + rdata.by_id("A_c_C1_l_C2_u_C3_u")[-1]
    + rdata.by_id("A_c_C1_u_C2_l_C3_u")[-1]
)
m2 = (
    rdata.by_id("A_c_C1_l_C2_l_C3_u")[-1]
    + rdata.by_id("A_c_C1_u_C2_l_C3_l")[-1]
    + rdata.by_id("A_c_C1_l_C2_u_C3_l")[-1]
)
m3 = rdata.by_id("A_c_C1_l_C2_l_C3_l")[-1]
total = m0 + m1 + m2 + m3

from numpy.testing import assert_array_almost_equal_nulp

assert_array_almost_equal_nulp(m0 / total, mid_corrected[0, -1])
assert_array_almost_equal_nulp(m1 / total, mid_corrected[1, -1])
assert_array_almost_equal_nulp(m2 / total, mid_corrected[2, -1])
assert_array_almost_equal_nulp(m3 / total, mid_corrected[3, -1])